In [ ]:
import numpy as np
from scipy import stats, signal
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import glob
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
file_list = glob.glob('/content/*.xlsx')  # Adjust path if necessary
file_list = sorted(file_list)

In [ ]:
real_spectras=pd.read_excel(file_list[0], engine='openpyxl', sheet_name='realSpectra').drop(columns=['Output', 'Real'])
synt_spectras=pd.read_excel(file_list[0], engine='openpyxl', sheet_name='realSpectra').drop(columns=['Output', 'Real'])

In [ ]:
real_spectras.head()

,467.3957824707031,467.5306091308594,467.6654052734375,467.8002014160156,467.9349975585938,468.0697631835938,468.2045288085938,468.3392639160156,468.4739990234375,468.6087036132812,...,708.0553588867188,708.1543579101562,708.2533569335938,708.3523559570312,708.4512939453125,708.5502319335938,708.649169921875,708.748046875,708.846923828125,708.9457397460938
0,772.00,738.50,767.50,744.00,774.50,747.50,751.00,766.50,769.50,741.00,...,1193.50,1134.50,1194.00,1185.50,1158.0,1137.50,1194.00,1147.00,1211.50,1167.50
1,641.25,624.50,641.00,633.50,631.75,649.25,622.00,649.00,638.00,644.75,...,815.00,805.50,777.50,760.75,791.0,796.50,818.00,795.50,782.50,814.75
2,683.25,709.50,669.25,708.00,703.75,694.25,691.00,701.75,721.25,706.75,...,946.00,929.25,936.00,1002.75,922.0,951.25,961.50,972.00,954.25,932.25
3,679.00,690.75,710.25,672.75,715.00,713.25,710.25,660.25,711.75,687.25,...,1030.75,1023.75,995.75,1043.00,1008.0,1001.50,983.25,1056.00,1032.75,977.25
4,666.25,659.25,677.75,704.00,671.25,689.00,688.50,679.50,677.25,679.25,...,875.50,898.25,885.00,902.25,909.5,881.00,886.00,868.75,893.00,889.25


In [ ]:
synt_spectras=shuffle(synt_spectras)

In [ ]:
# Preprocessing pipeline
def preprocess(spectra):
    spectra = np.asarray(spectra)
    min_val = np.min(spectra, axis=1, keepdims=True)
    max_val = np.max(spectra, axis=1, keepdims=True)
    denom = max_val - min_val
    denom[denom == 0] = 1e-8
    return (spectra - min_val) / denom

real_spectra = preprocess(real_spectras)
synthetic_set = preprocess(synt_spectras)

def calculate_metrics(spectra, reference=None):
    metrics = {'RMSE': [], 'Mean': [], 'Chi2': [], 'Frequency': []}

    if reference is None:
        reference = spectra.mean(axis=0)

    for s in spectra:
        # RMSE (vs reference spectrum)
        rmse = np.sqrt(mean_squared_error(reference, s))
        metrics['RMSE'].append(rmse)

        metrics['Mean'].append(np.mean(s))

        # KL Divergence (vs reference spectrum)
        s_norm = s / s.sum() if s.sum() > 0 else np.zeros_like(s)
        r_norm = reference / reference.sum() if reference.sum() > 0 else np.zeros_like(reference)

        # Add small constant to avoid division by zero
        s_norm = np.clip(s_norm, 1e-8, 1 - 1e-8)
        r_norm = np.clip(r_norm, 1e-8, 1 - 1e-8)

        kl_div = np.sum(r_norm * np.log(r_norm / s_norm))
        metrics['Chi2'].append(kl_div)

        # Frequency similarity (vs reference spectrum)
        fft_real = np.abs(np.fft.fft(reference))
        fft_synth = np.abs(np.fft.fft(s))
        cos_sim = np.dot(fft_real, fft_synth) / (np.linalg.norm(fft_real) * np.linalg.norm(fft_synth))
        metrics['Frequency'].append(cos_sim)
    return metrics

# Add composite metric calculation to compare_datasets
def compare_datasets(real_metrics, synth_metrics):
    # Calculate averages
    real_avg = {k: np.mean(v) for k,v in real_metrics.items()}
    synth_avg = {k: np.mean(v) for k,v in synth_metrics.items()}

    # Normalize metrics (scale to [0,1] range)
    metrics = ['RMSE', 'Chi2', 'Frequency', 'Mean']
    min_real = {m: min(real_metrics[m]) for m in metrics}
    max_real = {m: max(real_metrics[m]) for m in metrics}

    # Normalize real dataset metrics
    real_norm = {}
    for m in metrics:
        if max_real[m] - min_real[m] == 0:
            real_norm[m] = 0.5
        else:
            real_norm[m] = (real_avg[m] - min_real[m]) / (max_real[m] - min_real[m])

    synth_norm = {}
    for m in metrics:
        if max_real[m] - min_real[m] == 0:
            synth_norm[m] = 0.5
        else:
            synth_norm[m] = (synth_avg[m] - min_real[m]) / (max_real[m] - min_real[m])
    # Calculate composite metric (weighted average)
    weights = {'RMSE': 1, 'Chi2': 1, 'Frequency': 1, 'Mean': 1}  # Adjust weights as needed
    real_composite = sum(weights[m] * real_norm[m] for m in metrics)
    synth_composite = sum(weights[m] * synth_norm[m] for m in metrics)
    # Print comparison with composite metric
    print("Dataset Comparison:")
    print(f"{'Metric':<20} {'Real Avg':<10} {'Synth Avg':<10} {'Difference':<10}")
    print("-"*50)
    for metric in real_avg.keys():
        diff = synth_avg[metric] - real_avg[metric]
        print(f"{metric:<20} {real_avg[metric]:.6f} {synth_avg[metric]:.6f} {diff:.6f}")

    print("\nComposite Metrics:")
    print(f"Real Composite: {real_composite:.6f}")
    print(f"Synthetic Composite: {synth_composite:.6f}")
    print(f"Difference: {synth_composite - real_composite:.6f}")

if __name__ == "__main__":
    main()